# Texture feature value simulator
Copyright (c) 2016 Tetsuya Shinaji

This software and figures are released under the MIT License.

http://opensource.org/licenses/mit-license.php

In [10]:
import numpy as np
from matplotlib import pyplot as plt
import sys
sys.path.append('../')
from TextureAnalysis import GLCM_3D, GLSZM_3D, NGTDM, NGTDM_3D, GLHA
from ipywidgets import interact
from IPython.core.pylabtools import figsize
import pandas as pd
from IPython.display import display
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

def get_circle_img(r):
    y, x = np.ogrid[-64:128-64, -64:128-64]
    mask = x*x + y*y <= r*r
    array = np.zeros((1, 128, 128))
    array[0, mask] = 1
    return array, np.sqrt(x*x + y*y)

def get_half_grad_img():
    img = np.zeros((128, 128))
    img[:, 0:128] += np.arange(0, 128).reshape(1, -1)
    return img.reshape(1, 128, 128)

def get_grad_img():
    img = np.zeros((128, 128))
    img[0:128, :] += np.arange(0, 128).reshape(-1, 1)
    img[:, 0:128] += np.arange(0, 128).reshape(1, -1)
    return img.reshape(1, 128, 128)

def get_half_checkered_img():
    img = np.zeros((128, 128))
    img[:, 0:128:2] += np.arange(0, 128, 2).reshape(1, -1)
    img[:, 1:128:2] += np.arange(0, 128, 2).reshape(1, -1) + 63
    return img.reshape(1, 128, 128)

def get_checkered_img():
    img = np.zeros((128, 128))
    img[0:128:2, :] += np.arange(0, 128, 2).reshape(-1, 1)
    img[1:128:2, :] += np.arange(0, 128, 2).reshape(-1, 1) + 63
    img[:, 0:128:2] += np.arange(0, 128, 2).reshape(1, -1)
    img[:, 1:128:2] += np.arange(0, 128, 2).reshape(1, -1) + 63
    return img.reshape(1, 128, 128) / 2

%matplotlib inline

## GLCM: Gray-Level Co-occurrence Matrix

![image](./imgs/GLCM.png)

In [12]:
figsize(10, 10)
@interact(ptn=["hgrad", "grad", "hchkd", "chkd"], heterogeneity=(0, 100, 1), )
def test_glcm_dist(ptn="grad", heterogeneity=0):
    rng_seed = 0 
    np.random.seed(rng_seed)
    #     base_img, distance = get_circle_img(size)
    if ptn == "grad":
        base_img = get_grad_img()
    elif ptn == "hgrad":
        base_img = get_half_grad_img()
    elif ptn == "chkd":
        base_img = get_checkered_img()
    elif ptn == "hchkd":
        base_img = get_half_checkered_img()        
           
    if heterogeneity > 0:
        vmin = base_img.min()
        vmax = base_img.max()
        example_img =  np.random.normal(
            0, (vmax-vmin)/100*heterogeneity,size=base_img.shape) + base_img
    else:
        example_img = np.array(base_img)
    glcm = GLCM_3D(example_img, d=1, level_min=1, level_max=64, threshold=None)
    plt.subplot(121)
    plt.subplots_adjust(wspace=0.35)
    plt.title('Image')
    img = plt.imshow(glcm.img[0], vmin=0)
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Pixel value')
    plt.subplot(122)
    plt.title('GLCM')
    img = plt.imshow(glcm.matrix*100, extent=([1, 64, 64, 1]))
    plt.ylabel('Center pixel value')
    plt.xlabel('Neighbor pixel value')
    cbar = plt.colorbar(img, fraction=0.046, pad=0.04)
    cbar.set_label('Probability [%]')
    plt.show()
    features =pd.DataFrame.from_dict(glcm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

interactive(children=(Dropdown(description='ptn', index=1, options=('hgrad', 'grad', 'hchkd', 'chkd'), value='grad'), IntSlider(value=0, description='heterogeneity'), Output()), _dom_classes=('widget-interact',))

## GLSZM: Gray Level Size Zone Matrix

![image](./imgs/GLSZM.png)

In [13]:
figsize(10, 10)
glszm = None
@interact(ptn=["hgrad", "grad", "hchkd", "chkd"],
          heterogeneity=(0, 100, 1),)
def test_glszm_dist(ptn="grad", heterogeneity=0):
    rng_seed = 0 
    np.random.seed(rng_seed)
    #     base_img, distance = get_circle_img(size)
    if ptn == "grad":
        base_img = get_grad_img()
    elif ptn == "hgrad":
        base_img = get_half_grad_img()
    elif ptn == "chkd":
        base_img = get_checkered_img()
    elif ptn == "hchkd":
        base_img = get_half_checkered_img()        
           
    if heterogeneity > 0:
        vmin = base_img.min()
        vmax = base_img.max()
        example_img =  np.random.normal(
            0, (vmax-vmin)/100*heterogeneity,size=base_img.shape) + base_img
    else:
        example_img = np.array(base_img)

    global glszm
    glszm = GLSZM_3D(example_img, level_min=1, level_max=64, threshold=None)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('Image')
    img1 = ax1.imshow(glszm.img[0], vmin=0)
    cbar1 = fig.colorbar(img1,fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    ax2.set_title('GLSZM')
    img2 = ax2.imshow(glszm.matrix, 
                     aspect=glszm.matrix.shape[1]/glszm.matrix.shape[0],
                     extent=[glszm.min_zone_size-0.5, glszm.max_zone_size+0.5,
                             glszm.max_level, glszm.min_level],
                     )
    ax2.set_ylabel('Pixel value')
    ax2.set_xlabel('Size')
    cbar2 = fig.colorbar(img2, fraction=0.046, pad=0.04)
    cbar2.set_label('Number of areas')
    x = np.linspace(1, glszm.matrix.shape[1]+1, endpoint=True, num=5)
#     plt.xticks(x, x, rotation='vertical')
    plt.show()
    features =pd.DataFrame.from_dict(glszm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

interactive(children=(Dropdown(description='ptn', index=1, options=('hgrad', 'grad', 'hchkd', 'chkd'), value='grad'), IntSlider(value=0, description='heterogeneity'), Output()), _dom_classes=('widget-interact',))

## NGTDM: Neighbourhood Gray-Tone-Difference Matrix

![image](./imgs/NGTDM.png)

In [14]:
figsize(10, 10)
ngtdm = None
@interact(ptn=["hgrad", "grad", "hchkd", "chkd"],
          heterogeneity=(0, 100, 1))
def test_ngtdm_dist(ptn="grad", heterogeneity=0, scratch=False):
    rng_seed = 0 
    np.random.seed(rng_seed)
    #     base_img, distance = get_circle_img(size)
    if ptn == "grad":
        base_img = get_grad_img()
    elif ptn == "hgrad":
        base_img = get_half_grad_img()
    elif ptn == "chkd":
        base_img = get_checkered_img()
    elif ptn == "hchkd":
        base_img = get_half_checkered_img()        
           
    if heterogeneity > 0:
        vmin = base_img.min()
        vmax = base_img.max()
        example_img =  np.random.normal(
            0, (vmax-vmin)/100*heterogeneity,size=base_img.shape) + base_img
    else:
        example_img = np.array(base_img)

    if scratch:
        example_img[:, :, 63:65] = 128

    global ngtdm
    ngtdm = NGTDM(example_img[0], level_min=1, level_max=64, threshold=None)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('Image')
    img1 = ax1.imshow(ngtdm.img, vmin=0)
    cbar1 = fig.colorbar(img1,fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    ax2.set_title('NGTDM')
    img2 = ax2.imshow(ngtdm.s.reshape(ngtdm.s.size, 1), 
                      aspect=1/ngtdm.s.shape[0],
                      extent=[0, 1, ngtdm.level_max, ngtdm.level_min]
                     )
    ax2.set_ylabel('Pixel value')
    ax2.get_xaxis().set_visible(False)
    cbar2 = fig.colorbar(img2, fraction=0.046, pad=0.04)
    plt.show()
    features =pd.DataFrame.from_dict(ngtdm.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

interactive(children=(Dropdown(description='ptn', index=1, options=('hgrad', 'grad', 'hchkd', 'chkd'), value='grad'), IntSlider(value=0, description='heterogeneity'), Checkbox(value=False, description='scratch'), Output()), _dom_classes=('widget-interact',))

## GLHA: Gray Level Histogram Analysis

In [17]:
figsize(10, 10)
glha = None
@interact(ptn=["hgrad", "grad", "hchkd", "chkd"],
          heterogeneity=(0, 100, 1))
def test_glha_dist(ptn="grad", heterogeneity=0):
    rng_seed = 0 
    np.random.seed(rng_seed)
    #     base_img, distance = get_circle_img(size)
    if ptn == "grad":
        base_img = get_grad_img()
    elif ptn == "hgrad":
        base_img = get_half_grad_img()
    elif ptn == "chkd":
        base_img = get_checkered_img()
    elif ptn == "hchkd":
        base_img = get_half_checkered_img()        
           
    if heterogeneity > 0:
        vmin = base_img.min()
        vmax = base_img.max()
        example_img =  np.random.normal(
            0, (vmax-vmin)/100*heterogeneity,size=base_img.shape) + base_img
    else:
        example_img = np.array(base_img)

    global glha
    glha = GLHA(example_img, level_min=0, level_max=63, threshold=1)
    plt.subplots_adjust(wspace=0.5)
    fig = plt.gcf()
    ax1 = plt.subplot(121)
    ax1.set_title('A given image')
    img1 = ax1.imshow(glha.img[0], vmin=0)
    cbar1 = fig.colorbar(img1, fraction=0.046, pad=0.04)
    cbar1.set_label('Pixel value')
    ax2 = plt.subplot(122)
    hst = plt.hist(glha.img.flatten(), bins=64, range=[1, 64])
    plt.xlabel('Pixel value')
    plt.ylabel('Frequency')
    plt.title('Histogram')
    ax2.set_aspect(64/hst[0].max())
    plt.show()
    features =pd.DataFrame.from_dict(glha.features, orient="index")
    features.columns = ['Texture feature value']
    display(features)

interactive(children=(Dropdown(description='ptn', index=1, options=('hgrad', 'grad', 'hchkd', 'chkd'), value='grad'), IntSlider(value=0, description='heterogeneity'), Output()), _dom_classes=('widget-interact',))